<a href="https://colab.research.google.com/github/weimarquintero/UDEA-ai4eng-20242/blob/main/02%20-%20preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Creando el kaggle.json para descargar los datos directamente desde Kaggle.

In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

 99% 20.0M/20.1M [00:01<00:00, 24.8MB/s]
100% 20.1M/20.1M [00:01<00:00, 14.3MB/s]


Descomprimiendo el fichero y contando el numero de lineas de cada archivo.

In [3]:
!unzip udea*.zip > /dev/null

In [4]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  50135751 test.csv
   692501  10666231 118025055 train.csv
  1286075  15528571 172877479 total


Importando las librerias necesarias para trabajar.

In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


Cargando el archivo train.csv con pandas

In [9]:
df_train = pd.read_csv("train.csv")
print ("shape of loaded dataframe", df_train.shape)
df_train.head()

shape of loaded dataframe (692500, 12)


,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo


Validando las columnas de nuestro DF.

In [8]:
df_train.columns

Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_EDUCACIONMADRE', 'ESTU_PAGOMATRICULAPROPIO',
       'RENDIMIENTO_GLOBAL'],
      dtype='object')

Verificando cantidad de datos faltantes en las columnas.

In [15]:
f = df_train.isna().sum()
f[f!=0]

,0


Columnas con datos faltantes:

In [16]:
columns_with_missing_values = [
    'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
    'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET',
    'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE',
    'ESTU_PAGOMATRICULAPROPIO',
]

Rellenamos los valores faltantes en cada columna con la moda de cada una de ellas, luego verificamos que cada uno de los datos faltantes hallan sido completados de manera exitosa.

In [17]:
for column in columns_with_missing_values:
    mode_value = df_train[column].mode()[0]
    df_train[column].fillna(mode_value, inplace=True)

In [18]:
f = df_train.isna().sum()
f[f!=0]

,0
